In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import streamlit as st

In [2]:
  # Load data
df = pd.read_csv("city_day.csv")
df.head()


,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Ahmedabad,2015-01-01,NaN,NaN,0.92,18.22,17.15,NaN,0.92,27.64,133.36,0.00,0.02,0.00,NaN,NaN
1,Ahmedabad,2015-01-02,NaN,NaN,0.97,15.69,16.46,NaN,0.97,24.55,34.06,3.68,5.50,3.77,NaN,NaN
2,Ahmedabad,2015-01-03,NaN,NaN,17.40,19.30,29.70,NaN,17.40,29.07,30.70,6.80,16.40,2.25,NaN,NaN
3,Ahmedabad,2015-01-04,NaN,NaN,1.70,18.48,17.97,NaN,1.70,18.59,36.08,4.43,10.14,1.00,NaN,NaN
4,Ahmedabad,2015-01-05,NaN,NaN,22.10,21.42,37.76,NaN,22.10,39.33,39.31,7.01,18.89,2.78,NaN,NaN


In [3]:
import pandas as pd

# Remove rows where AQI or AQI_Bucket is NaN
clean_df = df.dropna(subset=['AQI', 'AQI_Bucket'])

# Group by AQI_Bucket and get min/max AQI
aqi_ranges = clean_df.groupby('AQI_Bucket')['AQI'].agg(['min', 'max']).reset_index()

print(aqi_ranges.sort_values('min'))

     AQI_Bucket    min     max
0          Good   13.0    50.0
3  Satisfactory   51.0   100.0
1      Moderate  101.0   200.0
2          Poor  201.0   300.0
5     Very Poor  301.0   400.0
4        Severe  401.0  2049.0


In [4]:
# Sort by AQI and check transitions
sorted_df = clean_df.sort_values('AQI')

# Find where AQI_Bucket changes
changes = sorted_df['AQI_Bucket'].ne(sorted_df['AQI_Bucket'].shift())
transition_points = sorted_df[changes]

print(transition_points[['AQI', 'AQI_Bucket']].head(20))

         AQI    AQI_Bucket
26329   13.0          Good
29444   51.0  Satisfactory
15300  101.0      Moderate
23350  201.0          Poor
240    301.0     Very Poor
26719  401.0        Severe


In [5]:
# Fill missing AQI_Bucket based on standard ranges
def assign_aqi_bucket(aqi):
    if pd.isna(aqi):
        return np.nan
    elif aqi <= 50:
        return "Good"
    elif aqi <= 100:
        return "Satisfactory"
    elif aqi <= 200:
        return "Moderate"
    elif aqi <= 300:
        return "Poor"
    elif aqi <= 400:
        return "Very Poor"
    elif aqi <= 500:
        return "Severe"
    else:
        return "Hazardous"

df['AQI_Bucket_Estimated'] = df['AQI'].apply(assign_aqi_bucket)